In [ ]:
from bicycleparameters.parameter_sets import Meijaard2007ParameterSet
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from data import bike_with_rider
from model import SteerControlModel

parameter_set = Meijaard2007ParameterSet(bike_with_rider, True)
model = SteerControlModel(parameter_set)

In [ ]:
%matplotlib widget

In [ ]:
from ipywidgets import interact

times = np.linspace(0.0, 10.0, num=500)
axes = model.plot_mode_simulations(times, v=6.0, kphi=0.0, kphidot=0.0)
fig = axes[0, 0].figure
fig.set_size_inches(10.0, 6.0)

def update_sim(v=6.0, kphi=0.0, kphidot=-50.0, kdelta=0.0, kdeltadot=0.0):
    results = model.simulate_modes(times, v=v,
                                   kphi=kphi, kphidot=kphidot,
                                   kdelta=kdelta, kdeltadot=kdeltadot)
    for i, row in enumerate(results):
        axes[i, 0].lines[0].set_data(times, results[i, :, 0])
        axes[i, 0].lines[1].set_data(times, results[i, :, 1])
        axes[i, 0].set_ylim((np.max([-30.0, np.min(results[i, :, :2])]),
                             np.min([30.0, np.max(results[i, :, :2])])))
        axes[i, 1].lines[0].set_data(times, results[i, :, 2])
        axes[i, 1].lines[1].set_data(times, results[i, :, 3])
        axes[i, 1].set_ylim((np.max([-30.0, np.min(results[i, :, 2:])]),
                             np.min([30.0, np.max(results[i, :, 2:])])))

interact(update_sim, v=(0.0, 10.0, 0.2),
         kphi=(-100.0, 20.0, 5), kphidot=(-150.0, 20.0, 5),
         kdelta=(-50.0, 50.0, 5), kdeltadot=(-50.0, 50.0, 5));

In [ ]:
speeds = np.array([0.6, 1.0, 1.4, 2.0, 3.0, 3.2, 4.0, 4.8, 6.4, 7, 10])
kphis = np.array([-80, -40, -25, -10, -5, -5, -5, -5, -5, -5, -5])
kphidots = np.array([-125, -80, -65, -50, -40, -35, -20, -10, -5, -5, -5]) - 5

In [ ]:
fig, ax = plt.subplots()
ax.plot(speeds, kphis, '.', label='kphi')
ax.plot(speeds, kphidots, '.', label='kphidot')
ax.legend()

In [ ]:
import scipy.optimize as spo

In [ ]:
f = lambda x, a, b, c: a*np.exp(b*x) + c
kphi_pars, _ = spo.curve_fit(f, speeds, kphis, p0=[-30, -2, -0.1])
print(kphi_pars)
ax.plot(speeds, f(speeds, *kphi_pars))
kphidot_pars, _ = spo.curve_fit(f, speeds, kphidots, p0=[-30, -2, -0.1])
ax.plot(speeds, f(speeds, *kphidot_pars))
print(kphidot_pars)

In [ ]:
speeds = np.linspace(0.0, 10.0, num=1000)
kphis = f(speeds, *kphi_pars) - 1
kphidots = f(speeds, *kphidot_pars)

In [ ]:
ax = model.plot_eigenvalue_parts(v=speeds, kphi=kphis, kphidot=kphidots)
ax = model.plot_eigenvalue_parts(ax=ax, colors=4*['black'], v=speeds)